In [73]:
from datetime import datetime
from dotenv import load_dotenv
import os
import pytz
import requests

import pandas as pd 

df = pd.read_pickle('Nonedf.pkl')

df = df.set_index('headers').T  # Set headers as index (headers) and transpose

print(df)

headers 1. From_Currency Code 2. From_Currency Name 3. To_Currency Code  \
value                     USD  United States Dollar                 AUD   

headers 4. To_Currency Name 5. Exchange Rate    6. Last Refreshed  \
value     Australian Dollar       1.57640000  2025-03-06 01:36:01   

headers 7. Time Zone 8. Bid Price 9. Ask Price  
value            UTC   1.57638000   1.57647000  


In [74]:
#i want to change last refreshed dates to AUS time

df.columns

Index(['1. From_Currency Code', '2. From_Currency Name', '3. To_Currency Code',
       '4. To_Currency Name', '5. Exchange Rate', '6. Last Refreshed',
       '7. Time Zone', '8. Bid Price', '9. Ask Price'],
      dtype='object', name='headers')

In [75]:
from datetime import datetime
from zoneinfo import ZoneInfo

x = pd.to_datetime(df['6. Last Refreshed'][0])

x



Timestamp('2025-03-06 01:36:01')

In [76]:
# Assuming x is a datetime object in UTC

x = x.replace(tzinfo=ZoneInfo('UTC'))

# Convert to Sydney time
australia_time = x.astimezone(ZoneInfo('Australia/Sydney'))

print(australia_time)

2025-03-06 12:36:01+11:00


In [77]:
df = df.reset_index(drop=True)


In [78]:
df['6. Last Refreshed'] = australia_time
df['7. Time Zone']  = 'UTC+10'

In [79]:
runtime_day = df['6. Last Refreshed'].dt.date

# Insert runtimedate at 0: likely to be partition index
df.insert(loc=0, column='Runtime_Day', value=runtime_day)
df

headers,Runtime_Day,1. From_Currency Code,2. From_Currency Name,3. To_Currency Code,4. To_Currency Name,5. Exchange Rate,6. Last Refreshed,7. Time Zone,8. Bid Price,9. Ask Price
0,2025-03-06,USD,United States Dollar,AUD,Australian Dollar,1.57640000,2025-03-06 12:36:01+11:00,UTC+10,1.57638000,1.57647000


In [80]:
str(runtime_day[0])

'2025-03-06'

In [81]:
df.columns

Index(['Runtime_Day', '1. From_Currency Code', '2. From_Currency Name',
       '3. To_Currency Code', '4. To_Currency Name', '5. Exchange Rate',
       '6. Last Refreshed', '7. Time Zone', '8. Bid Price', '9. Ask Price'],
      dtype='object', name='headers')

In [82]:
df['5. Exchange Rate'] = df['5. Exchange Rate'].astype(float)
df['8. Bid Price'] = df['8. Bid Price'].astype(float)
df['9. Ask Price'] = df['9. Ask Price'].astype(float)

In [83]:
# Save to Parquet file
df.to_parquet('output.parquet', index=False)

In [86]:
import pyarrow.parquet as pq

table = pq.read_table('output.parquet')
print(table.schema)

Runtime_Day: date32[day]
1. From_Currency Code: string
2. From_Currency Name: string
3. To_Currency Code: string
4. To_Currency Name: string
5. Exchange Rate: double
6. Last Refreshed: timestamp[us, tz=Australia/Sydney]
7. Time Zone: string
8. Bid Price: double
9. Ask Price: double
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 1434
